In [1]:
import os

from pandas import to_datetime
from dateutil.relativedelta import relativedelta

# Modify this to change incident, interaction, and arrests file names.
incidentName = 'INCALL'
interactionName = 'INTALL'
arrestsName = 'ARRESTS'
fileExtension = 'csv'
templateName = 'Template'

# Modify to change the date range.
# Dates are in the format YYYY-MM-DD
dateStartCurr = to_datetime("2024-06-10")
dateEndCurr = to_datetime("2024-07-07")
ytdCurr = to_datetime("2024-01-01")

# Modify this to get a CityWide or list of boroughs.
cw = True
boroList = ['PBBN','PBSI','PBMN']

# Modify this to change the output file names.
reportName = 'Force Stat Report'
presentationName = 'Overview'

currentDirectory = os.getcwd()
projectDirectory = os.path.dirname(currentDirectory)

def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

# Outputs the files in the /output folder of the project directory
outputDirectory = f'{projectDirectory}\\output'
reportPath = f'{projectDirectory}\\output\\{reportName}.xlsx'
presentationPath = f'{projectDirectory}\\output\\{presentationName}.pptx'

dateStartPre = dateStartCurr - relativedelta(years=1)
dateEndPre = dateEndCurr - relativedelta(years=1)
ytdPre = ytdCurr - relativedelta(years=1)

# Modify the name of the file and the find function will find it in the directory of the current project.
incallPath = find(f'{incidentName}.{fileExtension}', projectDirectory)
intallPath = find(f'{interactionName}.{fileExtension}', projectDirectory)
arrestsPath = find(f'{arrestsName}.{fileExtension}', projectDirectory)
templatePath = find(f'{templateName}.pptx', projectDirectory)

try:
    if not cw and not boroList:
        raise Exception("Please select CityWide or a borough.")

    if not os.path.exists(templatePath):
        print("Missing PowerPoint template file. Skipping that step...")

    if not os.path.exists(outputDirectory):
        os.mkdir(outputDirectory)

    if os.path.exists(reportPath):
        os.remove(reportPath)
        print("Removed old report spreadsheet.")
    else:
        print("Report does not exist. Creating new one.")

    if os.path.exists(presentationPath):
        os.remove(presentationPath)
        print("Removed old overview presentation.")
    else:
        print("Presentation does not exist. Creating new one.")

except PermissionError:
    raise Exception("Output file must be closed to run the program.")


Removed old report spreadsheet.
Removed old overview presentation.


In [2]:
import read

print('Reading files...')
incall, intall, arrests = read.readData(incallPath, intallPath, arrestsPath)
print('Done reading.\n')


Reading files...
Done reading.



In [3]:
import clean

print('Cleaning incidents...')
incall = clean.cleanINCALL(incall)
print('Cleaned incidents.\n')

print('Cleaning interactions...')
intall = clean.cleanINTALL(intall)
print('Cleaned interactions.\n')

print('Cleaning arrests...')
arrests = clean.cleanARRESTS(arrests)
print('Cleaned arrests.\n')

Cleaning incidents...
Cleaned incidents.

Cleaning interactions...
Cleaned interactions.

Cleaning arrests...
Cleaned arrests.



In [4]:
import filter

print('Setting date filters...')

(currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentInteractionYTD,
previousIncidentPeriod, previousIncidentYTD, previousInteractionPeriod, previousInteractionYTD,
currentArrestPeriod, currentArrestYTD, previousArrestPeriod, previousArrestYTD
) = filter.dates(incall,intall,arrests,dateStartCurr,dateEndCurr,dateStartPre,dateEndPre,ytdCurr,ytdPre)

print('Date filters set.\n')

Setting date filters...
Date filters set.



In [5]:
from pandas import DataFrame

dateDict = { 'Date Start:' : [dateStartCurr.strftime('%m/%d/%Y')], 'Date End:' : [dateEndCurr.strftime('%m/%d/%Y')]}
dateDataFrame = DataFrame(dateDict)
dateDataFrame.index = ['Date:']
dateDataFrame.index.name = 'Period'

In [6]:
import cwSummary
import cwBreakdown

cwTablesDict = {}

if cw == True:
    print('Building CityWide tables...')

    summaryDataFrames = cwSummary.periodSummary(currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentInteractionYTD,
    previousIncidentPeriod, previousIncidentYTD, previousInteractionPeriod, previousInteractionYTD,
    currentArrestPeriod, currentArrestYTD, previousArrestPeriod, previousArrestYTD)

    breakdownDataFrames = cwBreakdown.periodBreakdown(currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentArrestPeriod)

    summaryDataFrames = [dateDataFrame] + summaryDataFrames
    breakdownDataFrames = [dateDataFrame] + breakdownDataFrames

    cwTablesDict["CityWide Period Summary"] = summaryDataFrames
    cwTablesDict["CityWide Period Breakdown"] = breakdownDataFrames

    print("CityWide tables built.\n")

Building CityWide tables...
CityWide tables built.



In [7]:
import importlib
import boroSummary
import boroBreakdown
importlib.reload(boroBreakdown)

boroTablesDict = {}

for boro in boroList:
    print('Building borough tables...')

    summaryBoroDataFrames = boroSummary.boroSummary(boro, currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentInteractionYTD,
                    previousIncidentPeriod, previousIncidentYTD, previousInteractionPeriod, previousInteractionYTD,
                    currentArrestPeriod, currentArrestYTD, previousArrestPeriod, previousArrestYTD)
    
    breakdownBoroDataFrames = boroBreakdown.boroBreakdown(boro, currentIncidentPeriod, currentIncidentYTD, currentInteractionPeriod, currentArrestPeriod)

    summaryBoroDataFrames = [dateDataFrame] + summaryBoroDataFrames
    breakdownBoroDataFrames = [dateDataFrame] + breakdownBoroDataFrames

    boroTablesDict[f"{boro} Summmary"] = summaryBoroDataFrames
    boroTablesDict[f"{boro} Breakdown"] = breakdownBoroDataFrames

    print("Borough tables built.\n")

Building borough tables...
Borough tables built.



In [8]:
import overviewPpt

if cw==True and os.path.exists(templatePath):
    print("Filling overview presentation...")

    overviewPpt.fillPresentation(templatePath, presentationPath, summaryDataFrames, breakdownDataFrames, dateStartCurr, dateEndCurr)

    print(f"Output presentation to {presentationPath}")


Filling overview presentation...
Output presentation to c:\Users\robwo\Desktop\ForceStatGen\output\Overview.pptx


In [9]:
import output

outputDict = cwTablesDict | boroTablesDict

print("Outputting to Excel...")

output.outputToExcel(outputDict, reportPath)

print(f"Output to {reportPath}\n")

Outputting to Excel...
Output to c:\Users\robwo\Desktop\ForceStatGen\output\Force Stat Report.xlsx

